In [ ]:
pip install textstat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests                                                 #LOADING LIBRARIES
from bs4 import BeautifulSoup
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
import spacy
import codecs
from textstat.textstat import textstatistics
import pandas as pd
import numpy as np


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def data_ext(url):
  extracted_list=[]
  try:                                                                #DATA EXTRACTION
    page_url =url
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36'}
    rawpage = requests.get(page_url,headers=headers)
    soup = BeautifulSoup(rawpage.content, 'html5lib')
    extracted_text = soup.find_all('div',{'class':'td-post-content'})
    l1=(extracted_text)
    number_of_sentences = sent_tokenize(str(l1))
    # len(number_of_sentences)
    tknz=word_tokenize(str(l1))
    List = []
    for x in tknz:                                                    #CLEANING USING STOPWORDS
      List.append(x)
      len(List)
      li=[re.sub('(\d|\s|_|@|\.|\:|\/|\\|\//|\=,|\<|\>|\,|\[|\'|\"|\=|\(|\)|\”|\“|\’|\?|)', '', i) for i in List]
      while('' in li):
        li.remove('')
    f1 = open('/content/drive/MyDrive/20211030 Test Assignment/StopWords/StopWords_Auditor.txt')
    auditor = f1.read() 
    f2 = open('/content/drive/MyDrive/20211030 Test Assignment/StopWords/StopWords_Currencies.txt',encoding='ISO-8859-1')
    currencies = f2.read()
    f3 = open('/content/drive/MyDrive/20211030 Test Assignment/StopWords/StopWords_DatesandNumbers.txt')
    numbers = f3.read()
    f4 = open('/content/drive/MyDrive/20211030 Test Assignment/StopWords/StopWords_Generic.txt')
    generic = f4.read()
    f5 = open('/content/drive/MyDrive/20211030 Test Assignment/StopWords/StopWords_GenericLong.txt')
    generic_long = f5.read()
    f6 = open('/content/drive/MyDrive/20211030 Test Assignment/StopWords/StopWords_Geographic.txt')
    geographic = f6.read()
    f7 = open('/content/drive/MyDrive/20211030 Test Assignment/StopWords/StopWords_Names.txt')
    names = f7.read()
    clean_list=[]
    for i in li:
      if (i.lower() not  in auditor ) and (i.lower() not in currencies ) and (i not in numbers) and (i.lower() not in generic ) and (i.lower() not in generic_long ) and (i.lower() not in geographic ) and (i.lower() not in names):
        clean_list.append(i) 
    f8 = open('/content/drive/MyDrive/20211030 Test Assignment/MasterDictionary/positive-words.txt')
    positive = f8.read()
    f9 = open('/content/drive/MyDrive/20211030 Test Assignment/MasterDictionary/negative-words.txt',encoding='ISO-8859-1')
    negative = f9.read()
   

  except: 
    return 'page not found'


  extracted_list=[]
  try:                                                                #CREATING A DICTIONARY OF POSITIVE AND NEGATIVE WORDS
    positive_score=0
    for word in clean_list:
      if word in positive:
        positive_score=positive_score+1
    extracted_list.append(positive_score)
  except:
    extracted_list.append('NaN')
  
  try:
    negative_score=0
    for word in clean_list:
      if word in negative:
        negative_score=negative_score-1
        
        if negative_score<=0:
          negative_Score=negative_score*(-1)
    extracted_list.append(negative_Score)
  except: 
    extracted_list.append(negative_score)

  try:                                                                  #EXTRACTING DERIVED VARIABLES
    polarity_score = (positive_score - negative_Score)/ ((positive_score + negative_Score) + 0.000001)
    extracted_list.append(polarity_score)
  except:
    extracted_list.append('NaN')
  
  try:
    Subjectivity_Score = ( positive_score + negative_Score)/ ((len(clean_list)) + 0.000001)
    extracted_list.append(Subjectivity_Score)
  except: 
    extracted_list.append('NaN')
  
  try:
    Average_Sentence_Length = len(clean_list) / len(number_of_sentences)
    extracted_list.append(Average_Sentence_Length)
  except:
    extracted_list.append('NaN')
  
  try:
    diff_words_set = set()
    for i in clean_list:
      a=textstatistics().syllable_count(i)
      if a>2:
        diff_words_set.add(i)
    extracted_list.append(len(diff_words_set))
  except:
    extracted_list.append('NaN')

  try:
    Percentage_of_Complex_words = len(diff_words_set) / len(clean_list)
    extracted_list.append(Percentage_of_Complex_words)
  except:
    extracted_list.append('NaN')

  try:
    Fog_Index = 0.4 * (Average_Sentence_Length+ Percentage_of_Complex_words)
    extracted_list.append(Fog_Index)
  except:
    extracted_list.append('NaN')

  try:
    Average_Number_of_Words_Per_Sentence = len(List) / len(number_of_sentences)
    extracted_list.append(Average_Number_of_Words_Per_Sentence)
  except:
    extracted_list.append('NaN')

  try:
    Complex_Word_Count=len(diff_words_set)
    extracted_list.append(Complex_Word_Count)
  except:
    extracted_list.append('NaN')

  try:
    Word_Count=len(li)
    extracted_list.append(Word_Count)
  except:
    extracted_list.append('NaN')

  try:
    syllable_sum=0
    for s in clean_list:
      syllable=textstatistics().syllable_count(s)
      syllable_sum += syllable
    extracted_list.append(syllable_sum)
  except:
    extracted_list.append('NaN')
  
  try:
    count_i=extracted_text.count('I')
    count_we=extracted_text.count('we,')
    count_my=extracted_text.count('my,')
    count_ours=extracted_text.count('ours,')
    count_us=extracted_text.count('us')
    personal_pronouns_count=count_i+count_we+count_my+count_ours+count_us
    extracted_list.append(personal_pronouns_count)
  except:
    extracted_list.append('NaN')

  try:
    Sum_=sum(len(i) for i in clean_list)
    average_word_length=Sum_ / len(clean_list)
    extracted_list.append(average_word_length)
  except:
    extracted_list.append('NaN')

  return extracted_list


In [ ]:
df=pd.read_excel('/content/drive/MyDrive/20211030 Test Assignment/Input.xlsx')
for url in df['URL']:
  print(data_ext(url))

[147, 179, -0.0981595089013512, 0.29663330273281774, 21.134615384615383, 277, 0.25204731574158323, 8.554665080142787, 48.96153846153846, 277, 2023, 2647, 0, 8.272975432211101]
[125, 134, -0.03474903461486859, 0.4567901226511638, 8.338235294117647, 139, 0.24514991181657847, 3.4333540823736906, 25.779411764705884, 139, 1470, 1221, 0, 7.222222222222222]
[142, 172, -0.09554140096961337, 0.33727175044331714, 14.323076923076924, 241, 0.2588614393125671, 5.832775344955796, 35.323076923076925, 241, 1874, 2199, 0, 7.952738990332976]
[174, 146, 0.0874999997265625, 0.46579330354324117, 8.807692307692308, 146, 0.21251819505094613, 3.6080842010973018, 28.05128205128205, 146, 1774, 1443, 0, 7.069868995633188]
[178, 207, -0.07532467512902682, 0.3554939978250286, 15.926470588235293, 206, 0.1902123730378578, 6.446673184509262, 44.13235294117647, 206, 2171, 2538, 0, 8.395198522622346]
[129, 157, -0.09790209755978288, 0.34708737821955415, 17.53191489361702, 168, 0.20388349514563106, 7.094319355505061, 49

In [ ]:
output_df=pd.read_excel('/content/drive/MyDrive/20211030 Test Assignment/Output Data Structure.xlsx')
output_df.head()


,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,38.0,https://insights.blackcoffer.com/what-if-the-c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,40.0,https://insights.blackcoffer.com/will-machine-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,41.0,https://insights.blackcoffer.com/will-ai-repla...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,146.0,https://insights.blackcoffer.com/blockchain-fo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,147.0,https://insights.blackcoffer.com/the-future-of...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,148.0,https://insights.blackcoffer.com/big-data-anal...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112,149.0,https://insights.blackcoffer.com/business-anal...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
output_df=pd.read_excel('/content/drive/MyDrive/20211030 Test Assignment/Output Data Structure.xlsx')
pos=[]
neg=[]
pol=[]
Subject=[]
Ave=[] 
Percentage=[]
FogIndex=[]
Average_Number_of_WordsPerSentence=[]
Complex_WordCount=[]
WordCount=[]
syllableSum=[]
personal_pronounscount=[]
average_wordlength=[]
for i in output_df['URL']:
  s=data_ext(i)
  pos.append(s[0])
  neg.append(s[1])
  pol.append(s[2])
  Subject.append(s[3])
  Ave.append(s[4])
  Percentage.append(s[5])
  FogIndex.append(s[6])
  Average_Number_of_WordsPerSentence.append(s[7])
  Complex_WordCount.append(s[8])
  WordCount.append(s[9])
  syllableSum.append(s[10])
  personal_pronounscount.append(s[11])
  average_wordlength.append(s[-1])


In [ ]:
output_df['POSITIVE SCORE']=pos
output_df['NEGATIVE SCORE']=neg
output_df['POLARITY SCORE']=pol
output_df['SUBJECTIVITY SCORE']=Subject
output_df['AVG SENTENCE LENGTH']=Ave
output_df['PERCENTAGE OF COMPLEX WORDS']=Percentage
output_df['FOG INDEX']=FogIndex
output_df['AVG NUMBER OF WORDS PER SENTENCE']=Average_Number_of_WordsPerSentence
output_df['COMPLEX WORD COUNT']=Complex_WordCount
output_df['WORD COUNT']=WordCount
output_df['SYLLABLE PER WORD']=syllableSum
output_df['PERSONAL PRONOUNS']=personal_pronounscount
output_df['AVG WORD LENGTH']=average_wordlength

In [ ]:
ext_file=pd.ExcelWriter('Blackcoffer.xlsx')
output_df.to_excel(ext_file)
ext_file.save()

In [ ]:
data=pd.read_excel('/content/Blackcoffer.xlsx')
data

,Unnamed: 0,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,0,37,https://insights.blackcoffer.com/ai-in-healthc...,147,179,-0.098160,0.296633,21.134615,277,0.252047,8.554665,48.961538,277,2023,2647,8.272975
1,1,38,https://insights.blackcoffer.com/what-if-the-c...,125,134,-0.034749,0.456790,8.338235,139,0.245150,3.433354,25.779412,139,1470,1221,7.222222
2,2,39,https://insights.blackcoffer.com/what-jobs-wil...,142,172,-0.095541,0.337272,14.323077,241,0.258861,5.832775,35.323077,241,1874,2199,7.952739
3,3,40,https://insights.blackcoffer.com/will-machine-...,174,146,0.087500,0.465793,8.807692,146,0.212518,3.608084,28.051282,146,1774,1443,7.069869
4,4,41,https://insights.blackcoffer.com/will-ai-repla...,178,207,-0.075325,0.355494,15.926471,206,0.190212,6.446673,44.132353,206,2171,2538,8.395199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,109,146,https://insights.blackcoffer.com/blockchain-fo...,69,99,-0.178571,0.344969,12.815789,140,0.287474,5.241306,32.368421,140,1002,1080,7.887064
110,110,147,https://insights.blackcoffer.com/the-future-of...,163,183,-0.057803,0.309758,22.795918,166,0.148612,9.177812,59.673469,166,2029,2559,8.116383
111,111,148,https://insights.blackcoffer.com/big-data-anal...,97,167,-0.265152,0.441472,9.803279,133,0.222408,4.010275,23.885246,133,1221,1300,7.070234
112,112,149,https://insights.blackcoffer.com/business-anal...,71,75,-0.027397,0.302277,26.833333,122,0.252588,10.834369,69.166667,122,910,1259,8.875776
